In [1]:
import numpy as np

## Task 1 - FEM helper functions

Write four helper functions for FEM, in Python.

`delete_degrees_of_freedom_from_matrix`
   - Parameters:
     - `matrix`: 2D square array (N x N)
     - `deleted_degrees_of_freedom`: 1D array of integer indices (0-based)
   - Returns:
     - New square matrix (M x M) where M = N - number of deleted indices
     - Simultaneous removal of specified rows AND columns
     - Invalid indices should result in error

`rotation_matrix_2d`
   - Parameters:
     - `angle`: Floating point value in radians
   - Returns:
     - 2x2 rotation matrix
     - Positive angles must produce counter-clockwise rotation

`transformed_matrix`
   - Parameters:
     - `original_matrix`: Square array (n x n)
     - `transformation_matrix`: Square array (n x m)
   - Returns:
     - Transformed matrix: Square array (m x m)

`reorder_matrix`
   - Parameters:
     - `original_matrix`: Square array (n x n)
     - `reordering_of_the_degrees_of_freedom`: 1D array of integer indices (0-based)
   - Returns:
     - Reordered matrix: Square array (m x m)

**Implementation Requirements:**
1. Use NumPy for matrix operations
2. Include type hints
4. Validate input dimensions where appropriate
5. Maintain numerical stability
6. Optimize for performance with matrix operations


---

**Example 1:**

```python
import numpy as np

mat1 = np.arange(25).reshape((5,5))
print(mat1, '\n')

mat2 = reorder_matrix(mat1, np.array([4,3,1,0,2]))
print(mat2, '\n')

mat3  = delete_degrees_of_freedom_from_matrix(mat2, np.array([0,1,2]))
print(mat3, '\n')

rot = rotation_matrix_2d(np.pi/4)
print(rot, '\n')

print(transformed_matrix(mat3, rot))
```

*Output:*

```python
[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]] 

[[24 23 21 20 22]
 [19 18 16 15 17]
 [ 9  8  6  5  7]
 [ 4  3  1  0  2]
 [14 13 11 10 12]] 

[[ 0  2]
 [10 12]] 

[[ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]] 

[[ 1.200000e+01  2.000000e+00]
 [ 1.000000e+01 -3.616068e-16]]
```

---

**Example 2:**

```python
import numpy as np

mat1 = np.arange(16).reshape((4,4))
print(mat1, '\n')

mat2 = reorder_matrix(mat1, np.array([3,1,2]))
print(mat2, '\n')

mat3  = delete_degrees_of_freedom_from_matrix(mat2, np.array([0]))
print(mat3, '\n')

rot = rotation_matrix_2d(np.pi/4)
print(rot, '\n')
print(transformed_matrix(mat3, rot))
```

*Output:*

```python
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]] 

[[15 13 14]
 [ 7  5  6]
 [11  9 10]] 

[[ 5  6]
 [ 9 10]] 

[[ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]] 

[[ 1.50000000e+01  1.00000000e+00]
 [ 4.00000000e+00 -4.84675355e-16]]
```

---

In [57]:
"""code here"""
def delete_degrees_of_freedom_from_matrix(matrix,deleted_degrees_of_freedom):
    if np.max(deleted_degrees_of_freedom) <= np.shape(matrix)[0]:
        matrix = np.delete(matrix,deleted_degrees_of_freedom,axis=0) #excluding rows
        matrix = np.delete(matrix,deleted_degrees_of_freedom,axis=1) #excluding columns
        return(matrix)
    else:
        print('Invalid indices provided')
def rotation_matrix_2d(angle):
    return np.array([
        [np.cos(angle), -np.sin(angle)],
        [np.sin(angle),  np.cos(angle)]
    ])

def transformed_matrix(original_matrix,transformation_matrix):
    return transformation_matrix.T @ original_matrix @ transformation_matrix

def reorder_matrix(original_matrix,reordering_of_the_degrees_of_freedom):
    if original_matrix.shape[0] != original_matrix.shape[1]:
        raise ValueError("The input matrix must be square.")
    
    if sorted(reordering_of_the_degrees_of_freedom) != list(range(original_matrix.shape[0])):
        raise ValueError("index_vector must be a permutation of row/column indices.")
    
    return original_matrix[np.ix_(reordering_of_the_degrees_of_freedom, reordering_of_the_degrees_of_freedom)]

In [60]:
m = np.array([[15,13,14],[7,5,6],[11,9,10]])
m_del = delete_degrees_of_freedom_from_matrix(m,np.array([0]))
print(m)

rot = rotation_matrix_2d(np.pi/4)
print(rot)

print(transformed_matrix(m_del,rot))

mat1 = np.arange(25).reshape((5,5))
print(mat1, '\n')

mat2 = reorder_matrix(mat1, np.array([4,3,1,0,2]))
print(mat2, '\n')



[[15 13 14]
 [ 7  5  6]
 [11  9 10]]
[[ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]]
[[ 1.50000000e+01  1.00000000e+00]
 [ 4.00000000e+00 -4.84675355e-16]]
[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]] 

[[24 23 21 20 22]
 [19 18 16 15 17]
 [ 9  8  6  5  7]
 [ 4  3  1  0  2]
 [14 13 11 10 12]] 



## Task 2 - Element Assembly  

Create a function that inserts a smaller square matrix into specific positions of a larger square matrix through element-wise addition. The original global matrix must remain unchanged.

`insert_element`
   - Parameters:
     - `global_matrix`: 2D square array (N x N) serving as the base matrix
     - `element_matrix`: 2D square array (n x n) containing values to insert
     - `localization_array`: 1D array of indices (length n) specifying insertion locations
   - Returns:
     - New N x N matrix with element_matrix values added at specified positions
     - Identical indices in localization_array should accumulate values
     - Must handle cases where n ≤ N

**Implementation Requirements:**
1. Use NumPy array operations (no explicit loops)
3. Handle arbitrary matrix sizes where n ≤ N
4. Include type hints
5. Write complete docstrings
6. Optimize for memory efficiency

**Constraints:**
- element_matrix must be square
- localization_array length must match element_matrix dimensions
- Invalid indices may result in errors

---

**Example 3:**

```python
import numpy as np

element_matrix = np.array([[1, 2], [3, 4]]) * 1000
print(element_matrix, '\n')
localization_array = np.array([1, 3])

global_matrix = np.arange(25).reshape((5, 5))
print(insert_element(global_matrix, element_matrix, localization_array))
```

*Output:*

```python
[[1000 2000]
 [3000 4000]] 

[[   0    1    2    3    4]
 [   5 1006    7 2008    9]
 [  10   11   12   13   14]
 [  15 3016   17 4018   19]
 [  20   21   22   23   24]]
```

---

**Example 4:**

```python
import numpy as np

element_matrix = np.array([[1, 2], [3, 4]]) * -1000
print(element_matrix, '\n')
localization_array = np.array([0, 3])

global_matrix = np.arange(16).reshape((4, 4))
print(insert_element(global_matrix, element_matrix, localization_array))
```

*Output:*

```python
[[-1000 -2000]
 [-3000 -4000]] 

[[-1000     1     2 -1997]
 [    4     5     6     7]
 [    8     9    10    11]
 [-2988    13    14 -3985]]
```

---

In [61]:
"""code here"""
from typing import Union

def insert_element(global_matrix: Union[np.ndarray, list],
                   element_matrix: Union[np.ndarray, list],
                   localization_array: Union[np.ndarray, list]) -> np.ndarray:
    """
    Inserts a smaller square matrix into specific positions of a larger square matrix
    using element-wise addition, without modifying the original global matrix.

    Parameters:
    - global_matrix: 2D NumPy array (N x N), the base matrix.
    - element_matrix: 2D NumPy array (n x n), values to insert via addition.
    - localization_array: 1D NumPy array (length n), indices in the global matrix where
                          rows and columns of element_matrix are to be added.

    Returns:
    - A new NumPy array of shape (N x N) with values from element_matrix added at the
      positions specified by localization_array.

    Raises:
    - ValueError: If input shapes are inconsistent or if matrices are not square.
    """
    # Convert inputs to NumPy arrays
    global_matrix = np.asarray(global_matrix)
    element_matrix = np.asarray(element_matrix)
    localization_array = np.asarray(localization_array)

    # Check matrix shapes
    N = global_matrix.shape[0]
    if global_matrix.shape[0] != global_matrix.shape[1]:
        raise ValueError("global_matrix must be square.")
    if element_matrix.shape[0] != element_matrix.shape[1]:
        raise ValueError("element_matrix must be square.")
    if element_matrix.shape[0] != localization_array.shape[0]:
        raise ValueError("localization_array length must match element_matrix dimensions.")
    if np.any(localization_array < 0) or np.any(localization_array >= N):
        raise ValueError("localization_array contains invalid indices.")

    # Copy global matrix to avoid modifying the original
    result = global_matrix.copy()

    # Use advanced indexing with np.add.at for in-place accumulation
    row_idx = localization_array[:, None]
    col_idx = localization_array[None, :]
    np.add.at(result, (row_idx, col_idx), element_matrix)

    return result

In [62]:
element_matrix = np.array([[1, 2], [3, 4]]) * 1000
print(element_matrix, '\n')
localization_array = np.array([1, 3])

global_matrix = np.arange(25).reshape((5, 5))
print(insert_element(global_matrix, element_matrix, localization_array))

[[1000 2000]
 [3000 4000]] 

[[   0    1    2    3    4]
 [   5 1006    7 2008    9]
 [  10   11   12   13   14]
 [  15 3016   17 4018   19]
 [  20   21   22   23   24]]


## Task 3 - Global Assembly

Create a function that assembles a global matrix from multiple element matrices and their corresponding localization arrays.

`assemble_global_matrix`  
   - Parameters:  
     - `total_number_of_degrees_of_freedom` (int): size of the global assembled matrix N
     - `list_of_elements` (list[tuple[np.ndarray, np.ndarray]]): where each tuple contains:  
       - Element matrix: 2D square array (n x n) to be added, where n ≤ N 
       - Localization array: 1D array of indices (length n) specifying insertion positions  
   - Returns:  
     - Assembled global matrix (N x N) where N = max index in all localization arrays + 1  
     - Overlapping element contributions are summed  


---

**Example 5:**

```python
element1 = np.array([[1, 2], [3, 4]])
print(element1, '\n')
loc1 = np.array([0, 1])

element2 = np.array([[10, 20], [30, 40]])
print(element2, '\n')
loc2 = np.array([1, 2])

element3 = np.array([[100]])
print(element3, '\n')
loc3 = np.array([3])

# Assemble global matrix
global_matrix = assemble_global_matrix(4, [
    (element1, loc1),
    (element2, loc2),
    (element3, loc3)
])

print("Assembled Global Matrix:")
print(global_matrix)
```

*Output:*

```python
[[1 2]
 [3 4]] 

[[10 20]
 [30 40]] 

[[100]] 

Assembled Global Matrix:
[[  1.   2.   0.   0.]
 [  3.  14.  20.   0.]
 [  0.  30.  40.   0.]
 [  0.   0.   0. 100.]]
```

---

**Example 6:**

```python
element1 = np.arange(9).reshape((3,3))
print(element1, '\n')
loc1 = np.array([0, 1, 3])

element2 = np.array([[10, 20], [30, 40]])
print(element2, '\n')
loc2 = np.array([4, 2])

element3 = np.eye(2)
print(element3, '\n')
loc3 = np.array([1, 4])

# Assemble global matrix
global_matrix = assemble_global_matrix(5, [
    (element1, loc1),
    (element2, loc2),
    (element3, loc3)
])

print("Assembled Global Matrix:")
print(global_matrix)
```

*Output:*

```python
[[0 1 2]
 [3 4 5]
 [6 7 8]] 

[[10 20]
 [30 40]] 

[[1. 0.]
 [0. 1.]] 

Assembled Global Matrix:
[[ 0.  1.  0.  2.  0.]
 [ 3.  5.  0.  5.  0.]
 [ 0.  0. 40.  0. 30.]
 [ 6.  7.  0.  8.  0.]
 [ 0.  0. 20.  0. 11.]]
```

In [ ]:
"""code here"""
from typing import List, Tuple

def assemble_global_matrix(
    total_number_of_degrees_of_freedom: int,
    list_of_elements: List[Tuple[np.ndarray, np.ndarray]]
) -> np.ndarray:
    N = total_number_of_degrees_of_freedom
    global_matrix = np.zeros((N, N))

    for element_matrix, localization_array in list_of_elements:
        element_matrix = np.asarray(element_matrix)
        localization_array = np.asarray(localization_array)

        # Validate shapes
        n = element_matrix.shape[0]
        if element_matrix.shape[0] != element_matrix.shape[1]:
            raise ValueError("Element matrix must be square.")
        if localization_array.shape[0] != n:
            raise ValueError("Localization array length must match element matrix size.")
        if np.any(localization_array < 0) or np.any(localization_array >= N):
            raise ValueError("Localization array contains out-of-bounds indices.")

        # Generate index grids and accumulate
        row_idx = localization_array[:, None]
        col_idx = localization_array[None, :]
        np.add.at(global_matrix, (row_idx, col_idx), element_matrix)

    return global_matrix

In [65]:
element1 = np.array([[1, 2], [3, 4]])
print(element1, '\n')
loc1 = np.array([0, 1])

element2 = np.array([[10, 20], [30, 40]])
print(element2, '\n')
loc2 = np.array([1, 2])

element3 = np.array([[100]])
print(element3, '\n')
loc3 = np.array([3])

# Assemble global matrix
global_matrix = assemble_global_matrix(4, [
    (element1, loc1),
    (element2, loc2),
    (element3, loc3)
])

print("Assembled Global Matrix:")
print(global_matrix)

[[1 2]
 [3 4]] 

[[10 20]
 [30 40]] 

[[100]] 

Assembled Global Matrix:
[[  1.   2.   0.   0.]
 [  3.  14.  20.   0.]
 [  0.  30.  40.   0.]
 [  0.   0.   0. 100.]]
